In [49]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

In [10]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
tickers_input = input("Enter the stock tickers in your portfolio, separated by commas (e.g., AAPL, MSFT, GOOGL): ")
tickers = [ticker.strip().upper() for ticker in tickers_input.split(',')]
end_date = datetime.now()
start_date = end_date - timedelta(days=365)


data = {}
for ticker in tickers:
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    data[ticker] = stock_data['Adj Close']  

df = pd.DataFrame(data)


df.fillna(method='ffill', inplace=True)  # Forward fill for NaN values
df.dropna(inplace=True)  # Drop remaining NaNs if any


daily_returns = df.pct_change().dropna()


risk_free_rate = 0  #Can be adjusteed
daily_returns = df.pct_change().dropna()

trading_days = 252

sharpe_ratios = {}
for ticker in tickers:
    avg_daily_return = daily_returns[ticker].mean()
    std_dev_return = daily_returns[ticker].std()
    
    sharpe_ratio = ((avg_daily_return - risk_free_rate) / std_dev_return) * (trading_days ** 0.5)
    sharpe_ratios[ticker] = sharpe_ratio

# Fetch data for each ticker
data = {}
for ticker in tickers:
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    data[ticker] = stock_data

# Create a DataFrame of Adjusted Close prices
adj_close_df = pd.DataFrame({ticker: data[ticker]['Adj Close'] for ticker in tickers})

# Calculate daily returns
daily_returns = adj_close_df.pct_change().dropna()

# Additional Metrics Calculation
metrics = {}

for ticker in tickers:
    stock_data = data[ticker]
    
    # Calculate Beta
    # Compare with S&P 500 (using ^GSPC as an index)
    sp500 = yf.download('^GSPC', start=start_date, end=end_date)['Adj Close']
    sp500_returns = sp500.pct_change().dropna()
    beta = daily_returns[ticker].cov(sp500_returns) / sp500_returns.var()
    
    # Calculate Volatility (standard deviation of returns)
    volatility = daily_returns[ticker].std()
    
    # Moving Averages
    stock_data['50_day_MA'] = stock_data['Adj Close'].rolling(window=50).mean()
    stock_data['200_day_MA'] = stock_data['Adj Close'].rolling(window=200).mean()
    
    # Maximum Drawdown
    cumulative_return = (1 + daily_returns[ticker]).cumprod()
    peak = cumulative_return.cummax()
    drawdown = (cumulative_return - peak) / peak
    max_drawdown = drawdown.min()

    # Sortino Ratio (only penalizes for negative returns)
    negative_volatility = daily_returns[ticker][daily_returns[ticker] < 0].std()
    sortino_ratio = daily_returns[ticker].mean() / negative_volatility

    # Add metrics to dictionary
    metrics[ticker] = {
        "Sharpe Ratio": sharpe_ratio,
        "Beta": beta,
        "Volatility": volatility,
        "Max Drawdown": max_drawdown,
        "Sortino Ratio": sortino_ratio,
        "50-day MA": stock_data['50_day_MA'].iloc[-1],
        "200-day MA": stock_data['200_day_MA'].iloc[-1],
    }

# Convert metrics to DataFrame for easier viewing
metrics_df = pd.DataFrame(metrics).T
# Assign risk level based on combined metrics


def calculate_risk_level(sharpe_ratio, beta, volatility, max_drawdown, sortino_ratio, ma_short, ma_long):
    # Initialize score
    score = 0
    if sharpe_ratio >= 2.0:
        score += 1
    elif 1.0 <= sharpe_ratio < 2.0:
        score += 2
    elif 0.5 <= sharpe_ratio < 1.0:
        score += 3
    else:
        score += 4
    # Beta score
    if beta <= 0.5:
        score += 1
    elif 0.5 < beta <= 1.0:
        score += 2
    elif 1.0 < beta <= 1.5:
        score += 3
    else:
        score += 4

    # Volatility score
    if volatility < 0.02:
        score += 1
    elif 0.02 <= volatility < 0.04:
        score += 2
    elif 0.04 <= volatility < 0.06:
        score += 3
    else:
        score += 4

    # Max Drawdown score
    if max_drawdown > -0.10:
        score += 1
    elif -0.10 >= max_drawdown > -0.20:
        score += 2
    elif -0.20 >= max_drawdown > -0.30:
        score += 3
    else:
        score += 4

    # Sortino Ratio score
    if sortino_ratio >= 2.0:
        score += 1
    elif 1.5 <= sortino_ratio < 2.0:
        score += 2
    elif 1.0 <= sortino_ratio < 1.5:
        score += 3
    else:
        score += 4

    # Moving Averages score
    if ma_short > ma_long:
        score += 1  # Positive trend
    else:
        score += 3  # Negative trend

    # Map total score to risk level and recommendation
    if score <= 5:
        return "Very Low Risk"
    elif score <= 8:
        return "Low Risk"
    elif score <= 11:
        return "Moderately Low Risk"
    elif score <= 14:
        return "Low-Medium Risk"
    elif score == 15:
        return "Medium Risk"
    elif score <= 17:
        return "Medium-High Risk"
    elif score <= 19:
        return "High-Medium Risk"
    elif score <= 22:
        return "Moderately High Risk"
    elif score <= 24:
        return "High Risk"
    else:
        return "Very High Risk"

risk_assessment = {}
for ticker, metrics in metrics.items():
    risk_assessment[ticker] = calculate_risk_level(
        sharpe_ratio=metrics["Sharpe Ratio"],
        beta=metrics["Beta"],
        volatility=metrics["Volatility"],
        max_drawdown=metrics["Max Drawdown"],
        sortino_ratio=metrics["Sortino Ratio"],
        ma_short=metrics["50-day MA"],
        ma_long=metrics["200-day MA"]
    )

# Convert risk assessment to DataFrame for display
risk_assessment_df = pd.DataFrame.from_dict(risk_assessment, orient='index', columns=['Risk Level & Recommendation'])
risk_assessment_df.index.name = 'Name'

print("\nRisk Assessment for each stock:")
print(risk_assessment_df)
df

[*********************100%***********************]  1 of 1 completed
/var/folders/jh/5qgf89253j945pw6bjvtshgr0000gn/T/ipykernel_73732/2708386542.py:18: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)  # Forward fill for NaN values
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Risk Assessment for each stock:
     Risk Level & Recommendation
Name                            
AAPL             Low-Medium Risk


,AAPL
Date,
2023-11-09,181.481354
2023-11-10,185.695358
2023-11-13,184.101395
2023-11-14,186.731415
2023-11-15,187.299271
...,...
2024-11-01,222.910004
2024-11-04,222.009995
2024-11-05,223.449997
